In [5]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
from eumap.misc import find_files, nan_percentile, GoogleSheet, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.mapper import SpaceOverlay
import geopandas as gpd
from pathlib import Path
from minio import Minio
import rasterio
import pyproj
from shapely.geometry import Point
import requests
import warnings
warnings.filterwarnings('default')

# os.environ['PROJ_LIB'] = '/opt/conda/share/proj'
folder = '/mnt/primus/xuemeng_tmp_harbour/soc'
added_covar = 0 # wether overlay from scratch or from overlayed version

# /home/opengeohub/.local/bin

### check what needs to be overlayed

In [2]:
from shapely.geometry import Point
from geopandas import gpd  



data_file = f'{folder}/data/002.2_data_whole.csv'
data_file = f'{folder}/overlay_intermediate/temp_overlay_static.csv'

if added_covar:
    df = pd.read_csv(data_file, low_memory=False)
else:
    df = pd.read_csv(f'{folder}/data/000.2_soil.full_qa.controlled.csv', low_memory=False)

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

df4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
df3035 = df4326.to_crs("EPSG:3035")


### generate overlay links
- read in the files specified by Google Sheet
- convert the files into readable links for overlay

In [6]:
# read in potential usable overlay files
key_file = '/mnt/primus/xuemeng_tmp_harbour/soc/data/gaia-319808-913d36b5fca4.json'
url = 'https://docs.google.com/spreadsheets/d/1eIoPAvWM5jrhLrr25jwguAIR0YxOh3f5-CdXwpcOIz8/edit?usp=sharing'

gsheet = GoogleSheet(key_file, url)
covar = gsheet.covar

if added_covar:
    covar = covar.loc[covar['need update in overlay']=='1']


/home/opengeohub/.local/lib/python3.8/site-packages/gspread/auth.py:335: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  warnings.warn(


In [7]:
# function to generate file paths by year, and check if the urls are valid
def generate_overlay_path(row,year,filt=None):
    # check if the covariates resolution is 30m, otherwise use .vrt
    if row['spatial resolution'] != '30m':
        if '.vrt' not in row['path']:
            row_path = 'http://192.168.1.30:8333/ai4sh/vrt/' + row['path'].split('/')[-1] +'.vrt'
        else:
            row_path = row['path']
    else:
        row_path = row['path']
        
    # determine if static variable
    if row['temporal resolution'] == 'static':
        if '{perc}' in row_path:
            perc_list = row['perc'].split(',')
            output_paths = [row_path.replace('{perc}', perc) for perc in perc_list]
        else:
            output_paths = [row_path]
        
        return output_paths, output_paths
    
    if row['temporal resolution'] == 'long term':
        perc_list = row['perc'].split(',')
        output_paths = [row_path.replace('{perc}', perc) for perc in perc_list]
        return output_paths, output_paths
        
    # determine if the year is ahead of the availibility of the variable
    if year>int(row['end year']):
        year = int(row['end year'])
    if year<int(row['start year']):
        year = int(row['start year'])
        
    # determine if it's an annual variable or (bi)monthly variable
    if '{start_m}' not in row_path:
        output_paths = [row_path.replace('{year}',f'{int(year)}')]
    else:
        output_paths = []
        start_list = row['start_m'].split(', ')
        end_list = row['end_m'].split(', ')
        output_paths = [row_path.replace('{year}',f'{int(year)}').replace('{start_m}',start_list[i]).replace('{end_m}',end_list[i]) for i in range(len(end_list))]
    
    if '{perc}' in row_path:
        perc_list = row['perc'].split(',')
        output_paths = [p.replace('{perc}', perc) for p in output_paths for perc in perc_list]
        
    if (row['leap year'] == '1') & (year%4==0):
        output_paths = [p.replace('0228', '0229') if '0228' in p else p for p in output_paths]
    
    return output_paths, [i.replace(str(int(year)),'{year}') for i in output_paths]
    
def check_path(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        # Check if the status code is not 200 (OK). You might want to specifically check for 404 or other error codes.
        if response.status_code == 404:
            print(f"{url} returned HTTP 404 Not Found")
            return url
        elif response.status_code != 200: # URL is fine (HTTP 200)
            print(f"{url} returned HTTP {response.status_code}")
            return url
        return None  # also could be adapted to handle redirections (HTTP 3xx) separately if needed.
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {str(e)}")
        return url


In [9]:
sta_path = []
dym_path = []
namel = []

year = 2022
for index,row in covar.iterrows():
    paths, names = generate_overlay_path(row, year)
    if row['temporal resolution'] == 'static':
        sta_path.extend(paths)
        # namel.extend(names)
    else:
        dym_path.extend(paths)
        # namel.extend(names)
       
pathl = sta_path + dym_path    
for i in pathl:
    check_path(i)
    
sta_path = [ip.replace('.1.','.49.') for ip in sta_path]
sta_path = [Path(ip) for ip in sta_path]

dym_path = [ip.replace('.1.','.49.').replace('2000','{year}').replace('2001','{year}') for ip in dym_path]
# dym_path = [Path(ip) for ip in dym_path]

# pathl = sta_path + dym_path

In [ ]:
# from minio import Minio

# s3_config = {
#     'access_key': 'iwum9G1fEQ920lYV4ol9',
#     'secret_access_key': 'GMBME3Wsm8S7mBXw3U4CNWurkzWMqGZ0n2rXHggS0',
#     'host': '192.168.1.30:8333',
#     'bucket': 'ai4sh'
# }

#### Static overlay

In [13]:
from skmap.mapper import SpaceOverlay, SpaceTimeOverlay
ttprint(f'start overlaying for static, size: {len(df3035)}, column num: {len(sta_path)}')
temp = SpaceOverlay(
    points=df3035, 
    fn_layers=sta_path, verbose=True
).run()

temp = temp.drop(columns=['overlay_id', 'geometry'])
ttprint(f'finish overlaying for static, size: {len(temp)}, column num: {len(sta_path)}')
temp.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/temp_overlay_static.csv', index=False)

[15:59:39] start overlaying for static, size: 408668, column num: 268
[16:34:18] 1/268 bsf_glad.landsat.ard2.seasconv.longterm_p25_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:34:23] 2/268 bsf_glad.landsat.ard2.seasconv.longterm_p50_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:34:36] 3/268 bsf_glad.landsat.ard2.seasconv.longterm_p75_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:34:45] 4/268 bsf_glad.landsat.ard2.seasconv.yearly.m.theilslopes_m_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:36:02] 5/268 ndvi_glad.landsat.ard2.seasconv.longterm_p75_30m_s_20000101_20221231_eu_epsg.3035_v20231127
[16:36:02] 6/268 fapar_glad.landsat.ard2.seasconv.longterm_p75_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:36:04] 7/268 ndwi.gao_glad.landsat.ard2.seasconv.longterm_p50_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:36:04] 8/268 ndwi.gao_glad.landsat.ard2.seasconv.longterm_p25_30m_s_20000101_20221231_eu_epsg.3035_v20231218
[16:36:06] 9/268 ndwi.gao_glad.landsat.ard2

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

[19:20:17] finish overlaying for static, size: 408668, column num: 268


In [26]:
for year in np.arange(2000,2024):
    temp = df3035.loc[df3035['time']==year]
    if len(temp)==0:
        print(f'no data for year {year}, skip!')
    else:
        print(f'overlaying for year {year}, size {len(temp)}')
    dym_path = []
    namel = []
    for index,row in covar.iterrows():
        paths, names = generate_overlay_path(row, year)
        if row['temporal resolution'] != 'static':
            dym_path.extend(paths)
            namel.extend(names)
        
    namel = [ii.split('/')[-1][0:-4] for ii in namel]
    name_map = dict(zip(dym_path,namel))
    
    dym_path = [ip.replace('.1.','.49.') for ip in dym_path]
    dym_path = [Path(ip) for ip in dym_path]
    print(f'overlaying covariates number: {len(dym_path)}')
        
    temp = SpaceOverlay(
        points=temp, 
        fn_layers=dym_path, verbose=True
    ).run()
    
    temp = temp.rename(columnds = name_map)
    temp = temp.drop(columns=['overlay_id', 'geometry'])
    temp.to_csv(f'/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/dft_{int(year)}.csv',index=False)

overlaying for year 2000, size 28143
overlaying covariates number: 295
[22:55:12] 1/295 thermal_glad.landsat.seasconv_m_30m_s_20000101_20000228_eu_epsg.3035_v20231211
[22:55:12] 2/295 thermal_glad.landsat.seasconv_m_30m_s_20000901_20001031_eu_epsg.3035_v20231211
[22:55:12] 3/295 thermal_glad.landsat.seasconv_m_30m_s_20001101_20001231_eu_epsg.3035_v20231211
[22:55:12] 4/295 thermal_glad.landsat.seasconv_m_30m_s_20000501_20000630_eu_epsg.3035_v20231211
[22:55:12] 5/295 thermal_glad.landsat.seasconv_m_30m_s_20000301_20000430_eu_epsg.3035_v20231211
[22:55:12] 6/295 thermal_glad.landsat.seasconv_m_30m_s_20000701_20000831_eu_epsg.3035_v20231211
[22:55:13] 7/295 blue_glad.landsat.seasconv_m_30m_s_20000501_20000630_eu_epsg.3035_v20231211
[22:55:13] 8/295 blue_glad.landsat.seasconv_m_30m_s_20000701_20000831_eu_epsg.3035_v20231211
[22:55:13] 9/295 ndsi_glad.landast.ard2.seasconv_m_30m_s_20000501_20000630_eu_epsg.3035_v20231127
[22:55:13] 10/295 ndti_glad.landast.ard2.seasconv_m_30m_s_20000301_20

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

overlaying for year 2001, size 13017
overlaying covariates number: 295
[23:10:28] 1/295 thermal_glad.landsat.seasconv_m_30m_s_20010901_20011031_eu_epsg.3035_v20231211
[23:10:28] 2/295 thermal_glad.landsat.seasconv_m_30m_s_20010101_20010228_eu_epsg.3035_v20231211
[23:10:28] 3/295 thermal_glad.landsat.seasconv_m_30m_s_20010701_20010831_eu_epsg.3035_v20231211
[23:10:29] 4/295 thermal_glad.landsat.seasconv_m_30m_s_20011101_20011231_eu_epsg.3035_v20231211
[23:10:29] 5/295 thermal_glad.landsat.seasconv_m_30m_s_20010501_20010630_eu_epsg.3035_v20231211
[23:10:30] 6/295 thermal_glad.landsat.seasconv_m_30m_s_20010301_20010430_eu_epsg.3035_v20231211
[23:10:31] 7/295 blue_glad.landsat.seasconv_m_30m_s_20010501_20010630_eu_epsg.3035_v20231211
[23:10:32] 8/295 blue_glad.landsat.seasconv_m_30m_s_20010701_20010831_eu_epsg.3035_v20231211
[23:10:32] 9/295 green_glad.landsat.seasconv_m_30m_s_20010901_20011031_eu_epsg.3035_v20231211
[23:10:33] 10/295 green_glad.landsat.seasconv_m_30m_s_20010701_20010831_e

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

overlaying for year 2002, size 13878
overlaying covariates number: 295
[23:26:28] 1/295 thermal_glad.landsat.seasconv_m_30m_s_20020901_20021031_eu_epsg.3035_v20231211
[23:26:28] 2/295 thermal_glad.landsat.seasconv_m_30m_s_20020501_20020630_eu_epsg.3035_v20231211
[23:26:28] 3/295 thermal_glad.landsat.seasconv_m_30m_s_20020701_20020831_eu_epsg.3035_v20231211
[23:26:29] 4/295 thermal_glad.landsat.seasconv_m_30m_s_20021101_20021231_eu_epsg.3035_v20231211
[23:26:29] 5/295 thermal_glad.landsat.seasconv_m_30m_s_20020101_20020228_eu_epsg.3035_v20231211
[23:26:29] 6/295 thermal_glad.landsat.seasconv_m_30m_s_20020301_20020430_eu_epsg.3035_v20231211
[23:26:30] 7/295 blue_glad.landsat.seasconv_m_30m_s_20020301_20020430_eu_epsg.3035_v20231211
[23:26:30] 8/295 blue_glad.landsat.seasconv_m_30m_s_20020501_20020630_eu_epsg.3035_v20231211
[23:26:31] 9/295 blue_glad.landsat.seasconv_m_30m_s_20020701_20020831_eu_epsg.3035_v20231211
[23:26:31] 10/295 blue_glad.landsat.seasconv_m_30m_s_20020901_20021031_eu_

...

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

overlaying for year 2022, size 3294
overlaying covariates number: 295
[08:22:42] 1/295 blue_glad.landsat.seasconv_m_30m_s_20221101_20221231_eu_epsg.3035_v20231211
[08:22:42] 2/295 green_glad.landsat.seasconv_m_30m_s_20221101_20221231_eu_epsg.3035_v20231211
[08:22:43] 3/295 blue_glad.landsat.seasconv_m_30m_s_20220301_20220430_eu_epsg.3035_v20231211
[08:22:43] 4/295 blue_glad.landsat.seasconv_m_30m_s_20220501_20220630_eu_epsg.3035_v20231211
[08:22:43] 5/295 blue_glad.landsat.seasconv_m_30m_s_20220101_20220228_eu_epsg.3035_v20231211
[08:22:43] 6/295 green_glad.landsat.seasconv_m_30m_s_20220901_20221031_eu_epsg.3035_v20231211
[08:22:43] 7/295 green_glad.landsat.seasconv_m_30m_s_20220701_20220831_eu_epsg.3035_v20231211
[08:22:43] 8/295 red_glad.landsat.seasonv_m_30m_s_20220901_20221031_eu_epsg.3035_v20231211
[08:22:43] 9/295 blue_glad.landsat.seasconv_m_30m_s_20220701_20220831_eu_epsg.3035_v20231211
[08:22:43] 10/295 green_glad.landsat.seasconv_m_30m_s_20220101_20220228_eu_epsg.3035_v202312

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

no data for year 2023, skip!
overlaying covariates number: 295
[08:30:46] 1/295 ndvi_glad.landast.ard2.seasconv_m_30m_s_20221101_20221231_eu_epsg.3035_v20231127
[08:30:46] 2/295 swir1_glad.landsat.seasconv_m_30m_s_20220701_20220831_eu_epsg.3035_v20231211
[08:30:46] 3/295 swir1_glad.landsat.seasconv_m_30m_s_20220101_20220228_eu_epsg.3035_v20231211
[08:30:46] 4/295 thermal_glad.landsat.seasconv_m_30m_s_20221101_20221231_eu_epsg.3035_v20231211
[08:30:46] 5/295 swir2_glad.landsat.ard2.seasconv.m.yearly_p25_30m_s_20220101_20221231_eu_epsg.3035_v20231127
[08:30:46] 6/295 green_glad.landsat.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127
[08:30:46] 7/295 nos_glad.landsat.ard2.seasconv_m_30m_s_20220101_20221231_eu_epsg.3035_v20231127
[08:30:46] 8/295 blue_glad.landsat.seasconv_m_30m_s_20220101_20220228_eu_epsg.3035_v20231211
[08:30:46] 9/295 blue_glad.landsat.seasconv_m_30m_s_20220501_20220630_eu_epsg.3035_v20231211
[08:30:46] 10/295 ndvi_glad.landast.ard2.seasconv_m_

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

#### Dynamic layers

In [11]:
samp_points_f_sub = []
for _,rows in df3035.groupby('time'):
    samp_points_f_sub.append(rows.sample(int(rows.shape[0] * 0.002)))
samp_points_f_sub = pd.concat(samp_points_f_sub)
print(len(samp_points_f_sub))

# samp_points_f_sub = samp_points_f_sub.loc[samp_points_f_sub['time'].isin([2009,2010,2016])]
# print(len(samp_points_f_sub))

807


In [13]:
from skmap.mapper import SpaceOverlay, SpaceTimeOverlay

dff = samp_points_f_sub

dff['ref_date'] = pd.to_datetime(dff.time, format='%Y')
dff_overlaid = SpaceTimeOverlay(
    points=dff, col_date='ref_date', fn_layers=dym_path, verbose=True
).run()

ttprint(f'finish overlaying for static, size: {len(dff)}, column num: {len(dym_path)}') 

[15:11:19] Overlay 56 points from 2000 in 295 raster layers


/mnt/primus/xuemeng_tmp_harbour/soc/SOC-EU/code/src/scikit-map/skmap/mapper.py:141: DeprecationWarning: invalid escape sequence \*
  """


RasterioIOError: HTTP response code: 404

In [72]:
dff_overlaid.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/temp_overlay_mended.csv', index=False)

### assemble the overlayed annual datasets
- read in the overlayed soil data (with covariates) for each year
- combine them into a whole dataset

In [20]:
import random
def replacement():
    # Generate a random number between 30 and 43
    x = random.randint(30, 43)
    return f'http://192.168.49.{x}'
    

In [46]:

namem = {}
for year in np.arange(2000,2023,1):
    # print(year)
    dym_path = []
    namel = []
    for index,row in covar.iterrows():
        paths, names = generate_overlay_path(row, year)
        if row['temporal resolution'] != 'static':
            dym_path.extend(paths)
            namel.extend(names)
        
    namel = [ii.split('/')[-1][0:-4] for ii in namel]
    dym_path = [ii.replace('http://192.168.1.30',replacement()) for ii in dym_path]
    name_map = dict(zip(namel,dym_path))
    namem[str(year)] = name_map

sta_path = []
sta_name = []
for index,row in covar.iterrows():
    paths, names = generate_overlay_path(row, year)
    if row['temporal resolution'] == 'static':
        sta_path.extend(paths)
        sta_name.extend(names)
        
sta_name = [ii.split('/')[-1][0:-4] for ii in sta_name]
sta_path = [ii.replace('http://192.168.1.30',replacement()) for ii in sta_path]
name_map = dict(zip(sta_name,sta_path))

namem['static'] = name_map
    
import json
with open('/mnt/primus/xuemeng_tmp_harbour/soc/results/production_model/file_names_map.json', 'w') as file:
    json.dump(namem, file, indent=4)
    

In [46]:
tl = []
for year in np.arange(2000,2024,1):
    temp = pd.read_csv(f'/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/dft_{int(year)}.csv',low_memory = False)
    
    dym_path = []
    namel = []
    for index,row in covar.iterrows():
        paths, names = generate_overlay_path(row, year)
        if row['temporal resolution'] != 'static':
            dym_path.extend(paths)
            namel.extend(names)
        
    namel = [ii.split('/')[-1][0:-4] for ii in namel]
    name_dym = [ii.split('/')[-1][0:-4] for ii in dym_path]
    name_map = dict(zip(name_dym,namel))
    
    temp = temp.rename(columns = name_map)
    print(f'{year}, {len(temp.columns)}, {len(temp)}')
    tl.append(temp)
    
df_dym = pd.concat(tl)
print(f'whole 3035, cols:{len(df_dym.columns)}, size:{len(df_dym)}')


2000, 329, 28143
2001, 329, 13017
2002, 329, 13878
2003, 329, 15971
2004, 329, 7941
2005, 329, 15517
2006, 329, 14704
2007, 329, 23328
2008, 329, 12864
2009, 329, 27540
2010, 329, 14707
2011, 329, 18490
2012, 329, 27506
2013, 329, 21520
2014, 329, 17143
2015, 329, 33647
2016, 329, 16219
2017, 329, 15857
2018, 329, 31102
2019, 329, 15088
2020, 329, 10281
2021, 329, 10911
2022, 329, 3294
2023, 329, 0


In [49]:
df_sta = pd.read_csv('/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/temp_overlay_static.csv',low_memory=False)

In [68]:
# deal with duplicates
df_dym_no_duplicates = df_dym.drop_duplicates()
print(len(df_dym_no_duplicates))
df_sta_no_duplicates = df_sta.drop_duplicates()
print(len(df_sta_no_duplicates))


408467
408467


In [89]:
meta_list = ['id', 'lat', 'lon', 'time', 'hzn_top', 'hzn_btm', 'ref', 'nuts0', 'oc',
       'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay', 'silt', 'sand', 'caco3',
       'N', 'K', 'P', 'CEC', 'EC', 'oc_qa', 'N_qa', 'caco3_qa',
       'bulk_density_qa', 'clay_qa', 'silt_qa', 'sand_qa', 'ph_h2o_qa',
       'ph_cacl2_qa', 'P_qa', 'K_qa', 'EC_qa', 'CEC_qa']

df_merge = df_dym_no_duplicates.merge(df_sta_no_duplicates, how='inner', on = meta_list)
print(len(df_merge))

408467


In [ ]:
dff = df_merge

### fix the issue of cropland layer
crop = 'cropland.extent_glad.interpolate_p_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240604'
dff.loc[dff[crop].isna(),crop] = 0

## NDVI and NDWI
mend = pd.read_csv('/mnt/primus/xuemeng_tmp_harbour/soc/data/001.2_covar_overlayed.csv',low_memory =False)
mend  = mend[meta_list + ['accum.ndwi_glad.landsat.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240513','accum.ndvi_glad.landsat.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240513']]
mend_no_duplicates = mend.drop_duplicates()
print(len(mend_no_duplicates))

print(f'dff, {len(dff)} rows, {len(dff.columns)} cols')
dff = dff.drop(columns=['accum.ndwi_glad.landsat.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240513','accum.ndvi_glad.landsat.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240513'])
print(f'dff, {len(dff)} rows, {len(dff.columns)} cols')
dff = dff.merge(mend_no_duplicates, how='left', on=meta_list)
print(f'dff, {len(dff)} rows, {len(dff.columns)} cols')


In [94]:
### check covariates availability
drop_list = []
for col in dff.columns:
    if col in meta_list:
        continue
    if (dff[col].isna().sum()/len(dff))>0.02:
        print(col, dff[col].isna().sum())
        drop_list.append(col)
        
print(f'remove covariates with more than 2% data unavailable')
dff = dff.drop(columns = drop_list)

surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022 408467
fcf_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531.tif 196605
fgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531.tif 284212
lgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531.tif 284212
remove covariates with more than 2% data unavailable


In [95]:
print(f'dff, {len(dff)} rows, {len(dff.columns)} cols')


dff, 408467 rows, 592 cols


In [96]:
dff.to_csv(f'{folder}/data/001.3_covar_overlayed.30m.csv',index=False)

### Assign spatial blocking ID

In [57]:
df = pd.read_csv(f'{folder}/data/001.4_covar_overlayed.30m.tileid.csv',low_memory=False)
dff = df.drop(columns=['tile_id'])

In [67]:
# create a tiling system first
from eumap.parallel import TilingProcessing
from pathlib import Path
import rasterio
from shapely.geometry import box
import numpy as np
import pandas as pd
import geopandas as gpd

# # create tiling system
# raster_layer_fn = f'http://192.168.1.30:8333/ai4sh-landmasked/ndvi/ndvi_glad.landast.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127.tif'
# ds = rasterio.open(raster_layer_fn)
# tiles_size = ds.transform[0] * 1000 # 30m -> 30km
# tiling_system = TilingProcessing.generate_tiles(tiles_size, extent=ds.bounds, crs=ds.crs, raster_layer_fn=raster_layer_fn)
# tiling_system = tiling_system.to_crs("EPSG:4326")
# tiling_system = tiling_system[['tile_id','geometry']]
# tiling_system.to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg',  driver="GPKG")
# # tiling_system[tiling_system['raster_mode_count'] > 0].to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg.gpkg',  driver="GPKG")

from shapely.geometry import Point
tiles = gpd.read_file(f'{folder}/data/000_tiles.correct_epsg.3035.gpkg')
tiles = tiles.rename(columns={'id':'tile_id'})
# dff = pd.read_csv(f'{folder}/data/001_covar_overlayed.csv',low_memory=False)
gdf = gpd.GeoDataFrame(dff, geometry=gpd.points_from_xy(dff.lon, dff.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(tiles.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['geometry','index_right'])

print(f'dff, {len(joined_gdf)} rows, {len(joined_gdf.columns)} cols')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


dff, 408467 rows, 593 cols


In [68]:
# joined_gdf.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc/data/003_bedrock_whole.csv',index=False)
joined_gdf.to_csv(f'{folder}/data/001.4_covar_overlayed.30m.tileid.csv',index=False)

### epsg 4326

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# # epsg 4326 overlay
# co4326 = covar.loc[covar['epsg']=='4326']
# path_ori = [i.split('/')[-1][0:-4] for i in co4326['path']]

# for year in np.arange(2000,2024,1):
#     pathl = []
#     namel = []
#     for index,row in co4326.iterrows():
#         paths, names = generate_overlay_path(row, year)
#         pathl.extend(paths)
#         namel.extend(names)
#         # path4326.extend(generate_overlay_path(row,year))
        
# #     for i in pathl:
# #         check_path(i)
        
#     path_stem = [i.split('/')[-1][0:-4] for i in pathl]
#     namel = [i.split('/')[-1][0:-4] for i in namel]
#     name_mapping = dict(zip(path_stem,namel))
    
#     df_overlay = df4326.loc[df4326['time']==year]
#     if len(df_overlay)==0:
#         print(f'no data for year {year}')
#         continue
        
#     ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}')
#     pathl = [Path(ii) for ii in pathl]
#     dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
#     temp = dfo.run()
    
#     temp = temp.rename(columns=name_mapping)
#     temp=temp.drop(columns=['overlay_id'])
#     # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_4326.csv',index=False)
#     temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',index=False)
#     ttprint(f'finish overlaying for year {str(int(year))}')


# # read in 4326 datasets
# tl = []

# for year in np.arange(2000,2023,1):
#     temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',low_memory=False)
#     temp = temp.rename(columns=name_mapping)
#     print(f'{year}, {len(temp.columns)}, {len(temp)}')
#     tl.append(temp)

# df4326 = pd.concat(tl)
# print(f'whole 4326, {len(df4326.columns)}, {len(df4326)}')


# # merge to merge
# cols_list = df3035.columns.values.tolist()
# meta_list = ['lat', 'lon', 'oc', 'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay','silt', 'sand', 'caco3','N',
#              'K', 'P', 'CEC', 'EC', 'nuts0', 'time','hzn_top','hzn_btm','ref','sample_id']
# dff = pd.merge(df3035,df4326,on = meta_list, how='inner')
